In [1]:
library(tidyverse)
library(phyloseq)
library(ape)
library(microbiome)

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2020 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: ‘microbiome’


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:base’:

    transform




In [2]:
otu <- read.delim("../data/otu.GTDB98.txt", sep = "\t", header = TRUE)

In [3]:
tax_map <- read.delim('/mnt/btrfs/data/shogun/gtdb_95/r95.gtdb.tax', sep="\t", header=F)

In [4]:
colnames(tax_map) <- c('X.OTU.ID', 'tax')

In [5]:
otu <- tax_map %>% inner_join(otu, by = 'X.OTU.ID')

rownames(otu)<- otu$X.OTU.ID

In [6]:
taxa <- otu %>% 
  select(tax) %>% 
  separate(tax, c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species"),
           ";")  

In [7]:
#the output is a data frame of characters, and we need taxa to be recognized as factors
taxa<- taxa %>% 
  mutate_if(is.character, as.factor)

# taxa<- cbind(otu$X.OTU.ID, taxa)

# colnames(taxa)[1] <- "X.OTU.ID"

otu<- otu %>% 
  select(-X.OTU.ID, -tax)

# rownames(taxa) <- taxa$X.OTU.ID

# taxa <- taxa %>% 
#   select(-X.OTU.ID)

In [8]:
meta <- read.delim("../data/SampleID_map.txt", sep = "\t", header = TRUE)

rownames(meta) <- meta$X.SampleID

#and delete the first column because it is now redundant
meta<- meta %>% 
        select(-X.SampleID)

ids <- intersect(rownames(meta), colnames(otu))

In [9]:
# otu <- otu %>% rownames_to_column('rn') %>% filter(rn %in% ids)
# rownames(otu) <- otu$rnb
# otu <- otu %>% select(-rn)

In [10]:
otu <- otu %>% select(ids)

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(ids)` instead of `ids` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [11]:
meta <- meta %>% rownames_to_column('rn') %>% filter(rn %in% ids)
rownames(meta) <- meta$rn
meta <- meta %>% select(-rn)

In [12]:
# taxa <- taxa %>% rownames_to_column('rn') %>% filter(rn %in% ids)
# rownames(taxa) <- taxa$rn
# taxa <- taxa %>% select(-rn)

In [13]:
otu_mat<- as.matrix(otu)
tax_mat<- as.matrix(taxa)

#transform data to phyloseq objects
phylo_OTU<- otu_table(otu_mat, taxa_are_rows = TRUE)
phylo_TAX<- tax_table(tax_mat)
phylo_samples<- sample_data(meta)

tree <- read.tree("../data/shogun.tre")

#and put them in one object
phylo_object<- phyloseq(phylo_OTU, phylo_TAX, phylo_samples, tree_file=tree)

In [14]:
tree


Phylogenetic tree with 31910 tips and 31909 internal nodes.

Tip labels:
  GCF_002286985.1, GCF_003781945.1, GCF_000739595.1, GCF_002727125.1, GCF_000337915.1, GCF_004114995.1, ...
Node labels:
  , d__Archaea, 100, 74, 100, 100:p__Halobacteriota, ...

Rooted; includes branch lengths.

In [15]:
sample_names(phylo_object)    

[1] "MCT.f.0035" "MCT.f.0052" "MCT.f.0069" "MCT.f.0417" "MCT.f.0103"
  [6] "MCT.f.0120" "MCT.f.0137" "MCT.f.0154" "MCT.f.0222" "MCT.f.0256"
 [11] "MCT.f.0273" "MCT.f.0290" "MCT.f.0307" "MCT.f.0375" "MCT.f.0392"
 [16] "MCT.f.0426" "MCT.f.0443" "MCT.f.0460" "MCT.f.0494" "MCT.f.0511"
 [21] "MCT.f.0528" "MCT.f.0596" "MCT.f.0613" "MCT.f.0002" "MCT.f.0019"
 [26] "MCT.f.0070" "MCT.f.0104" "MCT.f.0121" "MCT.f.0138" "MCT.f.0155"
 [31] "MCT.f.0206" "MCT.f.0223" "MCT.f.0291" "MCT.f.0308" "MCT.f.0325"
 [36] "MCT.f.0359" "MCT.f.0376" "MCT.f.0393" "MCT.f.0410" "MCT.f.0427"
 [41] "MCT.f.0444" "MCT.f.0461" "MCT.f.0495" "MCT.f.0512" "MCT.f.0529"
 [46] "MCT.f.0546" "MCT.f.0580" "MCT.f.0597" "MCT.f.0614" "MCT.f.0003"
 [51] "MCT.f.0020" "MCT.f.0037" "MCT.f.0071" "MCT.f.0122" "MCT.f.0139"
 [56] "MCT.f.0156" "MCT.f.0224" "MCT.f.0241" "MCT.f.0258" "MCT.f.0275"
 [61] "MCT.f.0292" "MCT.f.0326" "MCT.f.0360" "MCT.f.0377" "MCT.f.0394"
 [66] "MCT.f.0411" "MCT.f.0428" "MCT.f.0445" "MCT.f.0462" "MCT.f.0513"
 [71] "MCT.f.0530" "MCT.f.0547" "MCT.f.0564" "MCT.f.0581" "MCT.f.0598"
 [76] "MCT.f.0615" "MCT.f.0004" "MCT.f.0038" "MCT.f.0055" "MCT.f.0072"
 [81] "MCT.f.0089" "MCT.f.0106" "MCT.f.0123" "MCT.f.0225" "MCT.f.0259"
 [86] "MCT.f.0293" "MCT.f.0310" "MCT.f.0327" "MCT.f.0344" "MCT.f.0361"
 [91] "MCT.f.0378" "MCT.f.0395" "MCT.f.0412" "MCT.f.0429" "MCT.f.0446"
 [96] "MCT.f.0463" "MCT.f.0514" "MCT.f.0531" "MCT.f.0565" "MCT.f.0582"
[101] "MCT.f.0599" "MCT.f.0616" "MCT.f.0039" "MCT.f.0107" "MCT.f.0124"
[106] "MCT.f.0141" "MCT.f.0158" "MCT.f.0209" "MCT.f.0226" "MCT.f.0243"
[111] "MCT.f.0260" "MCT.f.0277" "MCT.f.0294" "MCT.f.0311" "MCT.f.0345"
[116] "MCT.f.0362" "MCT.f.0379" "MCT.f.0396" "MCT.f.0430" "MCT.f.0447"
[121] "MCT.f.0464" "MCT.f.0481" "MCT.f.0532" "MCT.f.0549" "MCT.f.0566"
[126] "MCT.f.0583" "MCT.f.0600" "MCT.f.0006" "MCT.f.0057" "MCT.f.0074"
[131] "MCT.f.0091" "MCT.f.0108" "MCT.f.0142" "MCT.f.0159" "MCT.f.0227"
[136] "MCT.f.0261" "MCT.f.0295" "MCT.f.0312" "MCT.f.0329" "MCT.f.0346"
[141] "MCT.f.0363" "MCT.f.0380" "MCT.f.0397" "MCT.f.0414" "MCT.f.0431"
[146] "MCT.f.0482" "MCT.f.0516" "MCT.f.0533" "MCT.f.0567" "MCT.f.0601"
[151] "MCT.f.0618" "MCT.f.0007" "MCT.f.0058" "MCT.f.0075" "MCT.f.0109"
[156] "MCT.f.0126" "MCT.f.0143" "MCT.f.0160" "MCT.f.0211" "MCT.f.0228"
[161] "MCT.f.0245" "MCT.f.0313" "MCT.f.0330" "MCT.f.0347" "MCT.f.0364"
[166] "MCT.f.0381" "MCT.f.0415" "MCT.f.0432" "MCT.f.0517" "MCT.f.0534"
[171] "MCT.f.0551" "MCT.f.0568" "MCT.f.0602" "MCT.f.0008" "MCT.f.0042"
[176] "MCT.f.0059" "MCT.f.0076" "MCT.f.0144" "MCT.f.0161" "MCT.f.0229"
[181] "MCT.f.0263" "MCT.f.0297" "MCT.f.0314" "MCT.f.0331" "MCT.f.0348"
[186] "MCT.f.0365" "MCT.f.0382" "MCT.f.0416" "MCT.f.0433" "MCT.f.0450"
[191] "MCT.f.0467" "MCT.f.0535" "MCT.f.0552" "MCT.f.0586" "MCT.f.0603"
[196] "MCT.f.0620" "MCT.f.0009" "MCT.f.0043" "MCT.f.0060" "MCT.f.0077"
[201] "MCT.f.0094" "MCT.f.0111" "MCT.f.0128" "MCT.f.0145" "MCT.f.0162"
[206] "MCT.f.0230" "MCT.f.0247" "MCT.f.0264" "MCT.f.0298" "MCT.f.0315"
[211] "MCT.f.0349" "MCT.f.0366" "MCT.f.0383" "MCT.f.0400" "MCT.f.0086"
[216] "MCT.f.0434" "MCT.f.0519" "MCT.f.0553" "MCT.f.0570" "MCT.f.0587"
[221] "MCT.f.0604" "MCT.f.0621" "MCT.f.0010" "MCT.f.0044" "MCT.f.0061"
[226] "MCT.f.0095" "MCT.f.0112" "MCT.f.0146" "MCT.f.0163" "MCT.f.0214"
[231] "MCT.f.0231" "MCT.f.0265" "MCT.f.0299" "MCT.f.0316" "MCT.f.0350"
[236] "MCT.f.0384" "MCT.f.0435" "MCT.f.0486" "MCT.f.0520" "MCT.f.0554"
[241] "MCT.f.0605" "MCT.f.0622" "MCT.f.0011" "MCT.f.0045" "MCT.f.0096"
[246] "MCT.f.0113" "MCT.f.0130" "MCT.f.0215" "MCT.f.0232" "MCT.f.0249"
[251] "MCT.f.0266" "MCT.f.0300" "MCT.f.0334" "MCT.f.0351" "MCT.f.0368"
[256] "MCT.f.0385" "MCT.f.0402" "MCT.f.0419" "MCT.f.0453" "MCT.f.0470"
[261] "MCT.f.0538" "MCT.f.0555" "MCT.f.0606" "MCT.f.0623" "MCT.f.0012"
[266] "MCT.f.0063" "MCT.f.0080" "MCT.f.0114" "MCT.f.0148" "MCT.f.0216"
[271] "MCT.f.0233" "MCT.f.0250" "MCT.f.0267" "MCT.f.0301" "MCT.f.0318"
[276] "MCT.f.0335" "MCT.f.0352" "MCT.f.0369" "MCT.f.0403" "MCT.f.0437"
[281] "MCT.f.0522" "MCT.f.

In [16]:
phylo_object

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 268 taxa and 409 samples ]
sample_data() Sample Data:       [ 409 samples by 23 sample variables ]
tax_table()   Taxonomy Table:    [ 268 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 268 tips and 267 internal nodes ]

In [17]:
tax_table(phylo_object)

,Domain,Phylum,Class,Order,Family,Genus,Species
GCF_003697165.2,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia_coli
GCF_001729805.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Enterobacter,s__Enterobacter_roggenkampii
GCF_900200035.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Klebsiella_A,s__Klebsiella_A_grimontii
GCF_003315195.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Sutterella,s__Sutterella_wadsworthensis
GCF_000297775.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Sutterella,s__Sutterella_wadsworthensis_A
GCF_001182045.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Mesosutterella,s__Mesosutterella_massiliensis
GCF_000205025.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Parasutterella,s__Parasutterella_excrementihominis
GCA_000980495.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Parasutterella,s__Parasutterella_sp000980495
GCA_001917135.1,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__RF32,f__CAG-239,g__CAG-267,s__CAG-267_sp001917135
GCA_000432275.1,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__RF32,f__CAG-239,g__CAG-495,s__CAG-495_sp000432275


In [18]:
unifrac_weighted = UniFrac(phylo_object, weighted=T, fast=F)

Warning message in UniFrac(phylo_object, weighted = T, fast = F):
“Option `fast=FALSE` is deprecated. Only 'fast' UniFrac is supported in phyloseq.”


In [19]:
unifrac_unweighted = UniFrac(phylo_object, weighted=F, fast=F)

Warning message in UniFrac(phylo_object, weighted = F, fast = F):
“Option `fast=FALSE` is deprecated. Only 'fast' UniFrac is supported in phyloseq.”


In [20]:
write.table(as.matrix(unifrac_weighted), file = "../data/weighted.unifrac.txt", sep = "\t", quote = F, row.names = T)
write.table(as.matrix(unifrac_unweighted), file = "../data/unweighted.unifrac.txt", sep = "\t", quote = F, row.names = T)

In [21]:
alpha(phylo_object)

Observed richness

Other forms of richness

Diversity

Evenness

Dominance

Rarity



,observed,chao1,diversity_inverse_simpson,diversity_gini_simpson,diversity_shannon,diversity_fisher,diversity_coverage,evenness_camargo,evenness_pielou,evenness_simpson,⋯,dominance_dbp,dominance_dmn,dominance_absolute,dominance_relative,dominance_simpson,dominance_core_abundance,dominance_gini,rarity_log_modulo_skewness,rarity_low_abundance,rarity_rare_abundance
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MCT.f.0035,180,180.0000,11.013986,0.9092063,3.363357,19.04765,5,0.19522577,0.6476767,0.06118881,⋯,0.24373859,0.3705245,58994,0.24373859,0.09079365,0.7948339,0.8919612,2.057064,0.06824548,0.08342492
MCT.f.0052,187,187.0000,12.920237,0.9226020,3.378223,19.87822,5,0.35515395,0.6457949,0.06909217,⋯,0.20346070,0.3193503,49244,0.20346070,0.07739796,0.5187950,0.9010057,2.055345,0.05674869,0.28899898
MCT.f.0069,199,207.1667,11.212783,0.9108161,3.052713,21.31163,4,0.20278680,0.5767121,0.05634564,⋯,0.16757911,0.3295802,40558,0.16757911,0.08918392,0.5781723,0.9299944,2.060668,0.05206530,0.14699429
MCT.f.0417,220,248.1250,7.811913,0.8719904,2.967629,23.84761,3,0.10006641,0.5502103,0.03550869,⋯,0.26948805,0.4777406,65225,0.26948805,0.12800963,0.2051910,0.9218521,2.060647,0.06526383,0.71790210
MCT.f.0103,221,230.0000,19.128450,0.9477218,3.578039,23.96912,7,0.27412014,0.6628253,0.08655407,⋯,0.12354105,0.2294611,29902,0.12354105,0.05227815,0.4769812,0.8902007,2.057012,0.05058234,0.32708095
MCT.f.0120,194,203.0312,6.218211,0.8391820,2.727275,20.71287,2,0.10983280,0.5177198,0.03205263,⋯,0.30083212,0.5468285,72811,0.30083212,0.16081796,0.8666003,0.9354010,2.060563,0.05402591,0.06844549
MCT.f.0137,206,206.0294,19.789758,0.9494688,3.537941,22.15300,7,0.13510632,0.6640434,0.09606679,⋯,0.11686643,0.2190120,28286,0.11686643,0.05053119,0.6360474,0.8968720,2.057718,0.06679144,0.09104806
MCT.f.0154,235,245.2857,19.155332,0.9477952,3.692459,25.67965,7,0.15146544,0.6763259,0.08151205,⋯,0.12892723,0.2517147,31204,0.12892723,0.05220479,0.6160279,0.8705284,2.059313,0.05837341,0.13081957
MCT.f.0222,223,223.0000,22.194954,0.9549447,3.832436,24.21272,9,0.18652701,0.7087691,0.09952894,⋯,0.10967281,0.2147429,26544,0.10967281,0.04505529,0.4423561,0.8520681,2.058175,0.07482574,0.36083692


In [22]:
microbiome::alpha(phylo_object)

Observed richness

Other forms of richness

Diversity

Evenness

Dominance

Rarity



,observed,chao1,diversity_inverse_simpson,diversity_gini_simpson,diversity_shannon,diversity_fisher,diversity_coverage,evenness_camargo,evenness_pielou,evenness_simpson,⋯,dominance_dbp,dominance_dmn,dominance_absolute,dominance_relative,dominance_simpson,dominance_core_abundance,dominance_gini,rarity_log_modulo_skewness,rarity_low_abundance,rarity_rare_abundance
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MCT.f.0035,180,180.0000,11.013986,0.9092063,3.363357,19.04765,5,0.19522577,0.6476767,0.06118881,⋯,0.24373859,0.3705245,58994,0.24373859,0.09079365,0.7948339,0.8919612,2.057064,0.06824548,0.08342492
MCT.f.0052,187,187.0000,12.920237,0.9226020,3.378223,19.87822,5,0.35515395,0.6457949,0.06909217,⋯,0.20346070,0.3193503,49244,0.20346070,0.07739796,0.5187950,0.9010057,2.055345,0.05674869,0.28899898
MCT.f.0069,199,207.1667,11.212783,0.9108161,3.052713,21.31163,4,0.20278680,0.5767121,0.05634564,⋯,0.16757911,0.3295802,40558,0.16757911,0.08918392,0.5781723,0.9299944,2.060668,0.05206530,0.14699429
MCT.f.0417,220,248.1250,7.811913,0.8719904,2.967629,23.84761,3,0.10006641,0.5502103,0.03550869,⋯,0.26948805,0.4777406,65225,0.26948805,0.12800963,0.2051910,0.9218521,2.060647,0.06526383,0.71790210
MCT.f.0103,221,230.0000,19.128450,0.9477218,3.578039,23.96912,7,0.27412014,0.6628253,0.08655407,⋯,0.12354105,0.2294611,29902,0.12354105,0.05227815,0.4769812,0.8902007,2.057012,0.05058234,0.32708095
MCT.f.0120,194,203.0312,6.218211,0.8391820,2.727275,20.71287,2,0.10983280,0.5177198,0.03205263,⋯,0.30083212,0.5468285,72811,0.30083212,0.16081796,0.8666003,0.9354010,2.060563,0.05402591,0.06844549
MCT.f.0137,206,206.0294,19.789758,0.9494688,3.537941,22.15300,7,0.13510632,0.6640434,0.09606679,⋯,0.11686643,0.2190120,28286,0.11686643,0.05053119,0.6360474,0.8968720,2.057718,0.06679144,0.09104806
MCT.f.0154,235,245.2857,19.155332,0.9477952,3.692459,25.67965,7,0.15146544,0.6763259,0.08151205,⋯,0.12892723,0.2517147,31204,0.12892723,0.05220479,0.6160279,0.8705284,2.059313,0.05837341,0.13081957
MCT.f.0222,223,223.0000,22.194954,0.9549447,3.832436,24.21272,9,0.18652701,0.7087691,0.09952894,⋯,0.10967281,0.2147429,26544,0.10967281,0.04505529,0.4423561,0.8520681,2.058175,0.07482574,0.36083692


In [23]:
save.image("../data/environment.RData")